In [38]:
import pandas as pd

periods = 1000 #  for testing purposes, all is 1000

# Trading function

In [58]:
last_training = 621.38

def trading(df, n_ahead):
    adjusted_n_ahead = n_ahead-1 # bcuz lists start at 0
    df_adj = df.iloc[adjusted_n_ahead::63]

    latest_decision = 'hold'
    bank = 1000
    stock_count = 0

    for row, values in df_adj.iterrows():
        latest_close = last_training if values['time'] == 1 else df['Close'].iloc[row-adjusted_n_ahead-63]
        forecast = df['forecast'].iloc[row]
        expected = forecast - latest_close
        
        decision = 'hold'

        if expected > 0 and latest_decision != 'buy':
            decision = 'buy'
            latest_decision = 'buy'
            bank -= latest_close
            stock_count += 1
        
        if expected < 0 and latest_decision != 'sell' and stock_count > 0:
            decision = 'sell'
            latest_decision = 'sell'
            bank += latest_close
            stock_count -= 1
        
        print(decision, forecast, latest_close, bank)
    
    last_close = df['Close'].iloc[-63]
    if latest_decision == 'buy': # reverts decision if latest decision is buy
        bank += last_close

    buy_hold = last_close - last_training
    trading = bank - 1000

    print(f'trading: {round(trading, 2)}')
    print(f'buy&hold: {round(buy_hold, 2)} (buy: {last_training}, sell: {last_close})')

# ARMA-GARCH-trading

In [40]:
# reading data
ag_df = pd.read_csv('data/r_cross_val.csv', usecols=[0,1,2])
ag_df = ag_df[ag_df['time'] <= periods]

In [41]:
trading(ag_df, 1)

trading: 11.71
buy&hold: 11.71 (buy: 621.38, sell: 633.09)


In [42]:
trading(ag_df,3)

trading: 11.71
buy&hold: 11.71 (buy: 621.38, sell: 633.09)


In [43]:
trading(ag_df,5)

trading: 11.71
buy&hold: 11.71 (buy: 621.38, sell: 633.09)


In [44]:
trading(ag_df, 21)

trading: 11.71
buy&hold: 11.71 (buy: 621.38, sell: 633.09)


In [45]:
trading(ag_df, 63)

trading: 11.71
buy&hold: 11.71 (buy: 621.38, sell: 633.09)


# LSTM-trading

In [46]:
lstm_df = pd.read_csv('data/lstm_results.csv', usecols=[0,1,2])
lstm_df = lstm_df[lstm_df['time'] <= periods]

In [47]:
trading(lstm_df, 1)

trading: 11.71
buy&hold: 11.71 (buy: 621.38, sell: 633.09)


In [48]:
trading(lstm_df, 3)

trading: 11.71
buy&hold: 11.71 (buy: 621.38, sell: 633.09)


In [59]:
trading(lstm_df, 5)

buy 623.4436645507812 621.38 378.62
hold 624.1792602539062 622.77 378.62
hold 621.2070922851562 618.7 378.62
hold 618.8038330078125 617.12 378.62
hold 622.0387573242188 621.28 378.62
hold 622.6338500976562 622.31 378.62
hold 624.8314208984375 624.1 378.62
hold 625.654296875 624.98 378.62
hold 625.5830078125 624.87 378.62
hold 630.0834350585938 629.85 378.62
trading: 11.71
buy&hold: 11.71 (buy: 621.38, sell: 633.09)


In [60]:
trading(lstm_df, 21)

buy 627.6048583984375 621.38 378.62
hold 624.9934692382812 622.77 378.62
hold 626.387451171875 618.7 378.62
hold 620.3894653320312 617.12 378.62
sell 620.4441528320312 621.28 999.9
hold 620.3486328125 622.31 999.9
buy 624.7643432617188 624.1 375.79999999999995
hold 625.2144165039062 624.98 375.79999999999995
hold 625.1669921875 624.87 375.79999999999995
sell 627.8679809570312 629.85 1005.65
trading: 5.65
buy&hold: 11.71 (buy: 621.38, sell: 633.09)


In [51]:
trading(lstm_df, 63)

trading: 5.65
buy&hold: 11.71 (buy: 621.38, sell: 633.09)
